In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Dimensionality reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Set style
plt.style.use('seaborn-whitegrid')
sns.set_palette('husl')

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")
print("✅ Libraries imported successfully!")

## 1. Load and Explore Data

In [ ]:
# Load dental X-ray data
data = np.load('../../data/data/Dental-Panaromic-Autoencoder.npz')

print("Available keys in dataset:")
print(list(data.keys()))

# Load images (adjust key name based on actual data)
if 'images' in data.keys():
    images = data['images']
elif 'X' in data.keys():
    images = data['X']
else:
    # Use first available array
    key = list(data.keys())[0]
    images = data[key]
    print(f"Using key: {key}")

print(f"\nImage shape: {images.shape}")
print(f"Data type: {images.dtype}")
print(f"Value range: [{images.min():.2f}, {images.max():.2f}]")

In [ ]:
# Visualize sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(images[i], cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'Image {i+1}')

plt.suptitle('Sample Dental X-Ray Images', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 2. Data Preprocessing

In [ ]:
# Normalize images to [0, 1]
if images.max() > 1.0:
    images_normalized = images.astype('float32') / 255.0
else:
    images_normalized = images.astype('float32')

# Add channel dimension if needed
if len(images_normalized.shape) == 3:
    images_normalized = np.expand_dims(images_normalized, axis=-1)

print(f"Normalized shape: {images_normalized.shape}")
print(f"Value range: [{images_normalized.min():.2f}, {images_normalized.max():.2f}]")

# Split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(images_normalized, test_size=0.2, random_state=42)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

## 3. Model 1: Vanilla Autoencoder

In [ ]:
# Build Vanilla Autoencoder
img_shape = X_train.shape[1:]
latent_dim = 128

# Encoder
encoder_input = layers.Input(shape=img_shape)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Latent space
x = layers.Flatten()(x)
latent = layers.Dense(latent_dim, activation='relu', name='latent')(x)

encoder = models.Model(encoder_input, latent, name='encoder')

# Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(np.prod(img_shape[:-1]) // 64 * 128, activation='relu')(decoder_input)
x = layers.Reshape((img_shape[0]//8, img_shape[1]//8, 128))(x)
x = layers.Conv2DTranspose(128, (3, 3), activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', strides=2, padding='same')(x)
decoder_output = layers.Conv2D(img_shape[-1], (3, 3), activation='sigmoid', padding='same')(x)

decoder = models.Model(decoder_input, decoder_output, name='decoder')

# Full Autoencoder
autoencoder = models.Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')

autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("✅ Vanilla Autoencoder built")
autoencoder.summary()

In [ ]:
# Train Vanilla Autoencoder
history_vanilla = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

print("\n✅ Training complete!")

In [ ]:
# Visualize training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history_vanilla.history['loss'], label='Train Loss')
axes[0].plot(history_vanilla.history['val_loss'], label='Val Loss')
axes[0].set_title('Vanilla Autoencoder - Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history_vanilla.history['mae'], label='Train MAE')
axes[1].plot(history_vanilla.history['val_mae'], label='Val MAE')
axes[1].set_title('Vanilla Autoencoder - MAE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize reconstructions
n_samples = 5
test_samples = X_test[:n_samples]
reconstructions = autoencoder.predict(test_samples)

fig, axes = plt.subplots(2, n_samples, figsize=(15, 6))

for i in range(n_samples):
    # Original
    axes[0, i].imshow(test_samples[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original', fontweight='bold')
    
    # Reconstructed
    axes[1, i].imshow(reconstructions[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Reconstructed', fontweight='bold')

plt.suptitle('Vanilla Autoencoder - Reconstructions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Model 2: Variational Autoencoder (VAE)

In [ ]:
# VAE Sampling layer
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# VAE Encoder
encoder_input = layers.Input(shape=img_shape)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)

z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean, z_log_var])

vae_encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name='vae_encoder')

# VAE Decoder (reuse previous decoder)
vae = models.Model(encoder_input, decoder(z), name='vae')

# VAE Loss
reconstruction_loss = tf.reduce_mean(
    tf.reduce_sum(keras.losses.binary_crossentropy(encoder_input, vae(encoder_input)), axis=(1, 2))
)
kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
vae_loss = reconstruction_loss + kl_loss

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

print("✅ VAE built")
vae.summary()

In [ ]:
# Train VAE
history_vae = vae.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, X_test),
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

print("\n✅ VAE training complete!")

## 5. Model 3: Denoising Autoencoder

In [ ]:
# Add noise to training data
noise_factor = 0.3
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

print(f"Noisy training data shape: {X_train_noisy.shape}")

# Visualize noisy images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

for i in range(5):
    axes[0, i].imshow(X_test[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Clean', fontsize=12, fontweight='bold')
    
    axes[1, i].imshow(X_test_noisy[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_ylabel('Noisy', fontsize=12, fontweight='bold')

plt.suptitle('Clean vs Noisy Images', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Build Denoising Autoencoder (same architecture as vanilla)
denoising_ae = models.Model(encoder_input, decoder(encoder(encoder_input)), name='denoising_autoencoder')
denoising_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train on noisy input, clean output
history_denoising = denoising_ae.fit(
    X_train_noisy, X_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_noisy, X_test),
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

print("\n✅ Denoising Autoencoder training complete!")

In [ ]:
# Visualize denoising results
denoised = denoising_ae.predict(X_test_noisy[:5])

fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for i in range(5):
    axes[0, i].imshow(X_test[i].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Original', fontsize=12, fontweight='bold')
    
    axes[1, i].imshow(X_test_noisy[i].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_ylabel('Noisy', fontsize=12, fontweight='bold')
    
    axes[2, i].imshow(denoised[i].squeeze(), cmap='gray')
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_ylabel('Denoised', fontsize=12, fontweight='bold')

plt.suptitle('Denoising Autoencoder Results', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Latent Space Visualization

In [ ]:
# Extract latent representations
latent_representations = encoder.predict(X_test[:500])

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
latent_2d = tsne.fit_transform(latent_representations)

# Visualize
plt.figure(figsize=(10, 8))
plt.scatter(latent_2d[:, 0], latent_2d[:, 1], alpha=0.6, s=50, c=range(len(latent_2d)), cmap='viridis')
plt.colorbar(label='Sample Index')
plt.title('t-SNE Visualization of Latent Space', fontsize=16, fontweight='bold')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Anomaly Detection

In [ ]:
# Calculate reconstruction errors
reconstructions = autoencoder.predict(X_test)
reconstruction_errors = np.mean(np.square(X_test - reconstructions), axis=(1,2,3))

# Plot distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(reconstruction_errors, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Reconstruction Error', fontweight='bold')
plt.ylabel('Frequency', fontweight='bold')
plt.title('Distribution of Reconstruction Errors', fontsize=14, fontweight='bold')

# Mark threshold for anomalies
threshold = np.percentile(reconstruction_errors, 95)
plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2, label=f'95th percentile: {threshold:.4f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(range(len(reconstruction_errors)), reconstruction_errors, alpha=0.5)
plt.axhline(y=threshold, color='red', linestyle='--', linewidth=2, label='Anomaly Threshold')
plt.xlabel('Sample Index', fontweight='bold')
plt.ylabel('Reconstruction Error', fontweight='bold')
plt.title('Reconstruction Error by Sample', fontsize=14, fontweight='bold')
plt.legend()

plt.tight_layout()
plt.show()

anomalies = np.where(reconstruction_errors > threshold)[0]
print(f"\n🔍 Detected {len(anomalies)} potential anomalies ({len(anomalies)/len(X_test)*100:.1f}% of test set)")

## 8. Model Comparison

In [ ]:
# Evaluate all models
vanilla_loss = autoencoder.evaluate(X_test, X_test, verbose=0)
denoising_loss = denoising_ae.evaluate(X_test_noisy, X_test, verbose=0)

print("="*70)
print("MODEL COMPARISON")
print("="*70)
print(f"\n1. Vanilla Autoencoder:")
print(f"   Loss (MSE): {vanilla_loss[0]:.6f}")
print(f"   MAE: {vanilla_loss[1]:.6f}")

print(f"\n2. Variational Autoencoder (VAE):")
print(f"   Can generate new samples from latent space")
print(f"   Smoother latent space representation")

print(f"\n3. Denoising Autoencoder:")
print(f"   Loss (MSE): {denoising_loss[0]:.6f}")
print(f"   MAE: {denoising_loss[1]:.6f}")
print(f"   Robust to noise, learns cleaner features")

print("\n" + "="*70)

## 9. Summary and Applications

In [ ]:
print("="*70)
print("SESSION 12: AUTOENCODERS - SUMMARY")
print("="*70)

print("\n📊 DATASET:")
print(f"   • Total images: {len(images):,}")
print(f"   • Image shape: {img_shape}")
print(f"   • Latent dimension: {latent_dim}")

print("\n🎯 MODELS BUILT:")
print("   1. Vanilla Autoencoder - Basic reconstruction")
print("   2. Variational Autoencoder (VAE) - Generative model")
print("   3. Denoising Autoencoder - Noise-robust features")

print("\n🏥 MEDICAL IMAGING APPLICATIONS:")
print("   ✓ Image compression and reconstruction")
print("   ✓ Anomaly detection (abnormal X-rays)")
print("   ✓ Image denoising and enhancement")
print("   ✓ Feature extraction for diagnosis")
print("   ✓ Data augmentation (VAE generation)")

print("\n💡 KEY INSIGHTS:")
print(f"   • Compression ratio: {np.prod(img_shape)/latent_dim:.1f}:1")
print(f"   • Anomaly detection threshold: {threshold:.4f}")
print(f"   • Models can reconstruct dental X-rays with high fidelity")

print("\n🎓 SKILLS DEMONSTRATED:")
print("   ✅ Convolutional autoencoders")
print("   ✅ Variational inference (VAE)")
print("   ✅ Denoising techniques")
print("   ✅ Latent space visualization")
print("   ✅ Anomaly detection")
print("   ✅ Medical image processing")
print("   ✅ Model comparison")

print("\n" + "="*70)

In [ ]:
# Save models
autoencoder.save('../../milestone_3_deep_learning/session_12_autoencoders/models/vanilla_autoencoder.h5')
vae.save('../../milestone_3_deep_learning/session_12_autoencoders/models/vae.h5')
denoising_ae.save('../../milestone_3_deep_learning/session_12_autoencoders/models/denoising_autoencoder.h5')

print("✅ All models saved successfully!")